In [1]:
import numpy as np
import veloxchem as vlx

In [2]:
h2o_xyz = """3

O    0.000000000000        0.000000000000        0.000000000000
H    0.000000000000        0.740848095288        0.582094932012
H    0.000000000000       -0.740848095288        0.582094932012
"""

In [3]:
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.ostream.mute()

molecule = vlx.Molecule.read_xyz_string(h2o_xyz)
basis = vlx.MolecularBasis.read(molecule, "6-31g", ostream=None)

scf_results = scf_drv.compute(molecule, basis)

In [4]:
epsilon = scf_results["E_alpha"]  # orbital energies

In [5]:
erimo_drv = vlx.MOIntegralsDriver()  # ERI blocks in the MO basis

ovov = erimo_drv.compute_in_memory(molecule, basis, scf_drv.mol_orbs, "chem_ovov")
oovv = erimo_drv.compute_in_memory(molecule, basis, scf_drv.mol_orbs, "chem_oovv")

print("(ov|ov):", ovov.shape)
print("(oo|vv):", oovv.shape)

(ov|ov): (5, 8, 5, 8)
(oo|vv): (5, 5, 8, 8)


In [6]:
nocc = molecule.number_of_alpha_electrons()
norb = basis.get_dimension_of_basis()
nvirt = norb - nocc
ndim = nocc * nvirt

print("Number of occupied orbitals:", nocc)
print("Number of virtual orbitals :", nvirt)
print("Block dimension            :", ndim)

Number of occupied orbitals: 5
Number of virtual orbitals : 8
Block dimension            : 40


In [7]:
def delta(i, j):

    if i == j:
        delta = 1
    else:
        delta = 0

    return delta

In [8]:
E2 = np.zeros((2 * ndim, 2 * ndim))  # real valued

A = np.zeros((ndim, ndim))
B = np.zeros((ndim, ndim))

m = 0
for j in range(nocc):
    for b in range(nvirt):
        n = 0
        for i in range(nocc):
            for a in range(nvirt):
                A[m, n] = (
                    (epsilon[a + nocc] - epsilon[i]) * delta(a, b) * delta(i, j)
                    + 2 * ovov[j, b, i, a]
                    - oovv[j, i, b, a]
                )
                B[m, n] = -2 * ovov[j, b, i, a] + ovov[j, a, i, b]
                n += 1
        m += 1

E2[:ndim, :ndim] = A
E2[:ndim, ndim:] = B
E2[ndim:, :ndim] = B
E2[ndim:, ndim:] = A

In [9]:
core, homo, lumo = 0, 4, 5  # Python indexing of orbitals

print(f"{'1s-LUMO':>48s}{'HOMO-LUMO':>17s}")
print(" " * 41 + "=" * 25)
print(
    "Diagonal elements of A-block of E2   :"
    + f"{A[0,0]:14.8f}"
    + f"{A[(nocc-1)*nvirt,(nocc-1)*nvirt]:14.8f}"
)
print(
    "Associated orbital energy differences:"
    + f"{epsilon[lumo] - epsilon[core]:14.8f}"
    + f"{epsilon[lumo] - epsilon[homo]:14.8f}\n"
)

print(f"Maximum element of B-block of E2 : {np.max(B):12.8f}")

                                         1s-LUMO        HOMO-LUMO
Diagonal elements of A-block of E2   :   20.34024945    0.36669362
Associated orbital energy differences:   20.76493335    0.70960170

Maximum element of B-block of E2 :   0.21128138


In [10]:
lres_drv = vlx.LinearResponseEigenSolver()
lres_drv.ostream.mute()

E2_ref = lres_drv.get_e2(molecule, basis, scf_results)

In [11]:
np.testing.assert_allclose(E2, E2_ref, atol=1.0e-6)

In [12]:
dipole_mats = vlx.compute_electric_dipole_integrals(molecule, basis)

mu_z_ao = dipole_mats[2]

C = scf_results["C_alpha"]

mu_z = np.einsum("ap, ab, bq -> pq", C, mu_z_ao, C)

In [13]:
B1 = np.zeros(2 * ndim, dtype=complex)  # imaginary valued

g = []
for j in range(nocc):
    for b in range(nocc, norb):
        g.append(1j * np.sqrt(2) * mu_z[j, b])
g = np.array(g)

B1[:ndim] = g
B1[ndim:] = g.conj()

In [14]:
lrs_drv = vlx.LinearResponseSolver()
lrs_drv.ostream.mute()

B1_ref = lrs_drv.get_prop_grad(
    "electric_dipole", "z", molecule, basis, scf_drv.scf_tensors
)[0]

In [15]:
np.testing.assert_allclose(B1.imag, B1_ref, atol=1.0e-12)

In [16]:
N = -np.matmul(np.linalg.inv(E2), B1)

In [17]:
lrs_drv.a_operator = "electric dipole"
lrs_drv.b_operator = "electric dipole"

lrs_drv.a_components = ["z"]
lrs_drv.b_components = ["z"]

lrs_drv.frequencies = [0.0]

lrs_results = lrs_drv.compute(molecule, basis, scf_results)

In [18]:
N_ref = np.zeros(2 * ndim)

N_ref[:ndim] = -lrs_results["solutions"][("z", 0.0)].get_full_vector(1)
N_ref[ndim:] = lrs_results["solutions"][("z", 0.0)].get_full_vector(1)

In [19]:
np.testing.assert_allclose(N.imag, N_ref, atol=1.0e-6)

In [20]:
A1 = B1  # property gradient of the observable

print(f"Response function: {np.dot(A1.conj(), N).real:10.6f}")

Response function:  -4.298489


In [21]:
print(f"Reference value: {lrs_results['response_functions'][('z', 'z', 0.0)]:10.6f}")

Reference value:  -4.298489
